In [1]:
import re
import time
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from glob import glob

In [2]:
import pickle
import os

In [3]:
def get_var_name(var_value, namespace):
    """
    Given a variable value and namespace (e.g. globals() or locals()),
    returns the name of the variable as a string.
    """
    for name in namespace:
        if namespace[name] is var_value:
            return name
    return None

def remove_field(element):
    element.send_keys(Keys.CONTROL + 'A')
    element.send_keys(Keys.DELETE)

In [4]:
path = 'driver/chromedriver.exe'
driver = webdriver.Chrome(path)

In [5]:
driver.maximize_window() 
driver.minimize_window() 
driver.maximize_window() 
driver.switch_to.window(driver.current_window_handle)
driver.implicitly_wait(10)

In [6]:
driver.get('https://www.jobstreet.co.id/')
driver.implicitly_wait(2)

Search Bar

In [137]:
'''next
ios, java, php, react, node.js, android, .Net, 3D Studio Max, 3D Video, Accounting, Accurate, ACLS, Acting, Administrasi, Administrasi BPJS, Adobe Skills, Advokat, Alat Tes, Analisis Data, Android Development, Android Studio, Animator, Artificial Intelligence, Audit, AutoCad, Autodesk, Baby Sitter, Bahasa Inggris, Bahasa Jepang, Bahasa Mandarin, Bakery, Barista, Bartender, Berpikir Kritis, Blender, Brevet A/B, BTCLS, C#, C++, CAD, CDE, Chinese Food, Chiropractic, Cinematografi, Cisco, CMS, CNA, Codeigniter, Coffee Roasting, Content Creator, Content Marketing, Content Planning, Content Strategy, Content Writing, Cook Helper, Cooking, Copywriting, Corel Draw, Critical Thinking, CRM, CRNA, CSS, Data Analysis, Data Entry, Data Management, Data Visualisation, Data Warehouse, Database, Detail-oriented, Digital Marketing, Drafting, Driver Manual, Driver Matic, Drone, Dubbing, Editor, EDP, Ekspor-Impor, Electrical, ENPC, EQ, ERP, Estimator, Facebook Ads, Fiber Optik, Figma, Finance, Fokus, Forecasting, Fotografi, GELS, General Affair, Git, Gizi, Google Ads, Graphic Design, Grill, Ground Crew, Guide, Hairdo, Hairstylist, Helpdesk, Housekeeping, HRIS, HSE, HTML, Hubspot, ICU, Ilustrator, Indesign, Instalasi, Interior, Interview, IOS Development, IPCN, ISO, Japanese Food, Java, Java Script, K3, Kasir, Ketelitian, Ketenagakerjaan, Kolaborasi, Komunikasi, Konsultan Keuangan, Kreativitas, Kreativitas, Landscape, Laravel, Laundry, Legal Drafting, Legal Opinion, Litigasi, Machine Learning, Maintenance Hardware, Maintenance Software, Make Up artist, Maket, Manajemen Projek, Manufacture, Marketing, MatLab, Mechanical Electric, Mekanik Hidrolik, Mendengarkan Secara Aktif, Menjahit, Meta Ads, Microsoft Excel, Microsoft Office, Microsoft Word, Mikrotik, Montir, Motion Graphic, Multitasking, Music, MVC, My SQL, MYOB, Nail Art, Negosiasi, Networking/Jaringan, Node JS, OBS Studio, Operator, Operator Alat Berat, Operator Bulldozer, Operator Conveyer, Operator Excavator, Operator Forklift, Operator Mesin, Opertor Alat Berat, Origami, Otomasi, PABX, PALS , Paper Cutting, Pastry, Payroll, PCP/IP, Pengambilan Keputusan, Pengecatan, Pengelasan, Penyelesaian Masalah, Perencanaan & Penjadwalan, Perencanaan Budget, Perhitungan, Perundang-undangan, Photoshop, Pidato, PLC, Plumbing, Power BI, PPGD, PPIC, Premiere, Presentasi, Problem-solving, Product Knowledge, Programming, Psikolog, Psikotes, Public Speaking, QC, Racik Obat, Receptionist, Refleksiologi, Rekrutmen, Resep, Revit, Robotics, Sabar, Salesforce, SAP
'''
job_name = 'reports'

job_title = '-'.join(job_name.split())

driver.implicitly_wait(5)
driver.get(f'https://www.jobstreet.co.id/id/job-search/{job_title}-jobs-in-indonesia/')

jobs = []
job_num = 1
job_idx = 0

driver.implicitly_wait(1)
total_data = driver.find_element_by_css_selector('[data-automation="searchResultBar"]').text.split()[-2]

print('Total', total_data, 'data.')

Total 1.948 data.


In [138]:
print('No.'.center(5), '[Scraped, Page, Total]'.center(25), 'Job Title'.center(60), 'Info')

while True:
    job_cards = driver.find_elements_by_css_selector('[data-search-sol-meta]')

    try:
        curr_page = int(driver.find_element_by_id('pagination').get_attribute('value'))
        driver.implicitly_wait(2)
    except:
        curr_page = 1

    for job_card in job_cards:
        job = {}

        # scroll
        driver.execute_script('arguments[0].scrollIntoView({block: "end"});', job_card)

        try:
            title = job_card.text.split('\n')[0]
            title_class = driver.find_element_by_xpath(f"//span[contains(text(), '{title}')]").get_attribute('class').split()

            if len(title_class) > 1:
                job_idx += 1
                # print('❌❌❌ Continued!')
                continue
        except:
            pass

        try:
            job_card.find_element_by_css_selector('[rel]').click()
            driver.implicitly_wait(10)
        except:
            job_card.find_element_by_css_selector('[rel]').click()
            driver.implicitly_wait(10)
        

        '''scraping'''
        # job header: list of header information
        try:
            job_header = driver.find_element_by_css_selector('[data-automation="jobDetailsHeader"]').text.split('\n')[:3]
            driver.implicitly_wait(5)

            job['Title'], job['Company'], job['Area'] = job_header
        except:
            print('❌ JobTitle raise exception!')
            job['Title'], job['Company'], job['Area'] = None, None, None

        # job detail object
        try:
            job_detail = driver.find_element_by_css_selector('[data-automation="splitModeJobDetailsScrollWrapper"]')
            driver.implicitly_wait(5)

            try:
                # job description: list of description
                job_description = driver.find_element_by_css_selector('[data-automation="jobDescription"]').text.split('\n')
                driver.implicitly_wait(5)

                job['Description'] = job_description
            except:
                print('❌ JobDescription raise exception!')
                job['Description'] = None

            try:
                # additional information: list of information
                job_additional_information = job_detail.find_element_by_xpath('div/div[2]/div/div[1]/div/div[2]').text.split('\n')[1:][1::2]
                driver.implicitly_wait(5)

                job_additional_information = [i for i in job_additional_information if i.lower() != 'tidak terspesifikasi']

                job['Information'] = job_additional_information
            except:
                print('❌ JobInformation raise exception!')
                job['Information'] = None
        except:
            print('❌ JobDescription and JobInformation raise exception!')
            job['Description'], job['Information'] = None, None

        jobs.append(job)

        # log
        print(f"{job_idx}.".ljust(5), f"[{job_num}, {curr_page}, {total_data}]".center(25), f"{job['Title']}".center(60), 'scraped successfully ... 😊👍')

        job_num += 1
        job_idx += 1
    
    try:
        # scroll to "lanjut"
        driver.execute_script('arguments[0].scrollIntoView({block: "center"});', driver.find_element_by_xpath("//span[contains(text(), 'Lanjut')]"))
    except:
        print('> raise break when scroll <')
        break

    # go to the next page
    driver.find_element_by_xpath("//span[contains(text(), 'Lanjut')]").click()
    driver.implicitly_wait(10)

    next_page = int(driver.find_element_by_id('pagination').get_attribute('value'))

    if curr_page == next_page:
        print('> raise break <')
        break


 No.    [Scraped, Page, Total]                           Job Title                           Info
0.          [1, 1, 1.948]                            Costing Supervisor                      scraped successfully ... 😊👍
1.          [2, 1, 1.948]                        Account Payable Admin (AP)                  scraped successfully ... 😊👍
2.          [3, 1, 1.948]                   Funding Marketing (Semarang Branch)              scraped successfully ... 😊👍
3.          [4, 1, 1.948]                          Accounting & Tax Staff                    scraped successfully ... 😊👍
5.          [5, 1, 1.948]                     HSE Supervisor (PLN AMI Project)               scraped successfully ... 😊👍
6.          [6, 1, 1.948]                            Accounting Finance                      scraped successfully ... 😊👍
7.          [7, 1, 1.948]                         Finance Accounting Staff                   scraped successfully ... 😊👍
8.          [8, 1, 1.948]                             S

In [139]:
len(jobs)

1010

In [140]:
filename = 'jobstreet_' + ''.join(job_name.title().split())

with open(f'data/scraped/{filename}.pickle', 'wb') as file:
    pickle.dump(jobs, file)

In [141]:
filepaths = glob('data/scraped/jobstreet_*.pickle')

filenames = [filepath.split('.')[0].split('_')[-1].lower() for filepath in filepaths]

if ''.join(job_name.lower().split()) in filenames:
    print(job_name.upper(), 'saved!')

REPORTS saved!


In [474]:
# with open("data/scraped/jobstreet_Akuntansi.pickle", "rb") as f:
#     my_list = pickle.load(f)

# print(len(my_list))

3198


In [232]:
# body = driver.find_element_by_tag_name("body")
# body.send_keys(Keys.CONTROL + '-')

In [233]:
# job_search = driver.find_element_by_css_selector('#global-nav-typeahead > input')

# job_search.click()
# job_search.send_keys(Keys.CONTROL + 'a')
# job_search.send_keys(Keys.DELETE)
# job_search.send_keys('ios developer')

# location_search = driver.find_element_by_css_selector('#jobs-search-box-location-id-ember1360')

# location_search.click()
# location_search.send_keys(Keys.CONTROL + 'a')
# location_search.send_keys(Keys.DELETE)
# location_search.send_keys('indonesia')

# search_button = driver.find_element_by_xpath('//*[@id="global-nav-search"]/div/div[2]/button[1]')
# search_button.click()

# actions = ActionChains(driver)
# actions.send_keys(Keys.ENTER)
# actions.perform()